--Importación de librerias--

In [ ]:
#Importamos las librerías necesarias
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

--Definiendo la ruta y asignando el dataset--

In [ ]:
#Cargamos el dataset de Titanic
#URL del dataset de Titanic: https://www.kaggle.com/c/titanic/data
titanic_path = 'D:\\Hacking\\Python\\AI_Learning\\Aprendizaje_Supervisado\\Titanic_Survive\\titanic.csv'
titanic_data = pd.read_csv(titanic_path)

--Marcando las caracteristicas de entrenamiento del modelo y la variable objetivo--

In [ ]:
#Seleccionamos las caracteristicas que vamos a utilizar (incluyendo categoricas y numericas)
titanic_features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
#Seleccionamos la variable objetivo que vamos a predecir
titanic_target = 'Survived'

--Filtrado de datos--

In [ ]:
#Eliminamos filas con valores nulos en las caracteristicas y la variable objetivo
titanic_data = titanic_data[titanic_features + [titanic_target]].dropna()

--Asignacion de caracteristicas a las variables de entrenamiento y testeo--

In [ ]:
#One-hot encoding de las caracteristicas categoricas
X = pd.get_dummies(titanic_data[titanic_features])
y = titanic_data[titanic_target]

In [ ]:
#Dividimos los datos en dos conjuntos: entrenamiento y prueba
#test_size=0.2 significa que el 20% de los datos se utilizaran para probar el modelo
#random_state=42 asegura que la division sea reproducible
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=12)

--Definiendo y entrenando el XGBoost--

In [ ]:
#Defimos el XGBoost que vamos a utilizar
#El modelo XGBClassifier es utilizado para problemas de clasificacion
xgb_model = XGBClassifier(
    eval_metric='logloss',  #Metrica de evaluacion
    use_label_encoder=False,  #Evita el warning de label encoder
    random_state=12  #Semilla para la aleatoriedad
)

#Ajustamos el modelo a los datos
xgb_model.fit(train_X, train_y)

#Generamos una prediccion de los datos con el modelo
xgb_preds = xgb_model.predict(test_X)

--Evaluación del modelo--

In [ ]:
#Calculamos la precision del modelo
xgb_accuracy = accuracy_score(test_y, xgb_preds)

#Imprimimos la precisión del modelo
print(f"Precisión del XGBoost: {100 * xgb_accuracy:.2f}%")
#Imprimimos el reporte de clasificación
print(classification_report(test_y, xgb_preds))

--Analizando que variables han sido más importantes--

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Importancias originales del modelo
importances = xgb_model.feature_importances_
feature_names = train_X.columns

# Convertimos a DataFrame para manejarlo mejor
fi_df = pd.DataFrame({'feature': feature_names, 'importance': importances})

# Agrupar por nombre original (antes del "_")
fi_df['original_feature'] = fi_df['feature'].str.split('_').str[0]
fi_grouped = fi_df.groupby('original_feature')['importance'].sum().sort_values()

# Gráfico limpio
plt.figure(figsize=(8,5))
plt.barh(fi_grouped.index, fi_grouped.values)
plt.xlabel('Importancia')
plt.ylabel('Características')
plt.title('Importancia de las Características (Agrupadas)')
plt.show()

--Guardando los modelos entrenados--

In [ ]:
import joblib
#Guardamos los modelos ya entrenados
joblib.dump(xgb_model, 'xgb_model.pkl')


--Cargando los modelos entrenados--

In [ ]:
import joblib
#Para cargar los modelos guardados y usarlos posteriormente
xgb_model_loaded = joblib.load('xgb_model.pkl')


--Haciendo nuevas predicciones para ver el correcto funcionamiento del modelo--

In [ ]:
#Creamos un nuevo pasajero para predecir su supervivencia
new_passenger = pd.DataFrame([{
    'Pclass': 3,
    'Sex': 'male',
    'Age': 22,
    'SibSp': 1,
    'Parch': 0,
    'Fare': 7.25,
    'Embarked': 'S'
}])

--Tratamiento de los datos del nuevo pasajero--

In [ ]:
#Realizamos el one-hot encoding del nuevo pasajero
new_passenger_encoded = pd.get_dummies(new_passenger)

#Alineamos las columnas del nuevo pasajero con las del modelo
new_passenger_encoded = new_passenger_encoded.reindex(columns=train_X.columns, fill_value=0)

--Realizando la prediccion de supervivencia sobre los dos modelos--

In [ ]:
#Realizamos la prediccion de supervivencia
xgb_prediction = xgb_model_loaded.predict(new_passenger_encoded)

--Mostrando el resultado sobre la supervivencia del nuevo pasajero en los dos modelos--

In [ ]:
#Mostramos la prediccion de supervivencia
print(f"Sobrevive el pasajero (XGBoost): {'Sí' if xgb_prediction[0] == 1 else 'No'}")

--Mejorando la entrada de nuevos pasajeros--

In [ ]:
# Definimos una función para crear un nuevo pasajero con las características necesarias
def new_passenger(pclass, sex, age, sibsp, parch, fare, embarked, columns):
    
    # Creamos un DataFrame con las características del nuevo pasajero
    # Nos aseguramos que las columnas coincidan con las del modelo
    df = pd.DataFrame([{
        'Pclass': pclass,
        'Sex': sex,
        'Age': age,
        'SibSp': sibsp,
        'Parch': parch,
        'Fare': fare,
        'Embarked': embarked
    }])
    
    # Realizamos el one-hot encoding de las características categóricas
    xgb_encoded = pd.get_dummies(df)
    # Alineamos las columnas del nuevo pasajero con las del modelo
    # Esto asegura que todas las columnas necesarias estén presentes, incluso si algunas son 0
    xgb_encoded = xgb_encoded.reindex(columns=columns, fill_value=0)
    # Devolvemos el DataFrame codificado
    return xgb_encoded

--Añadiendo nuevo pasajero--

In [ ]:
new_passenger_encoded = new_passenger(pclass=3, sex='male', age=22, sibsp=1, parch=0, fare=7.25, embarked='S', columns=train_X.columns)

--Realizando la prediccion de supervivencia sobre los dos modelos--

In [ ]:
xgb_prediction = xgb_model_loaded.predict(new_passenger_encoded)


--Mostrando el resultado sobre la supervivencia del nuevo pasajero en los dos modelos--

In [ ]:
#Mostramos la prediccion de supervivencia
print(f"Sobrevive el pasajero (XGBoost): {'Sí' if xgb_prediction[0] == 1 else 'No'}")